# Introduction

In [1]:
import ipywebgl
import numpy as np

In [2]:
w = ipywebgl.GLViewer()
w

GLViewer()

In [3]:
w.clear_color(1,0.8,1,1)
w.clear()
w.render()

In [4]:
pr = w.create_program()
pr.compile(
"""#version 300 es
 
// an attribute is an input (in) to a vertex shader.
// It will receive data from a buffer
in vec4 a_position;
 
// all shaders have a main function
void main() {
 
  // gl_Position is a special variable a vertex shader
  // is responsible for setting
  gl_Position = a_position;
}
"""
,
"""#version 300 es
 
// fragment shaders don't have a default precision so we need
// to pick one. highp is a good default. It means "high precision"
precision highp float;
 
// we need to declare an output for the fragment shader
out vec4 outColor;
 
void main() {
  // Just set the output to a constant reddish-purple
  outColor = vec4(1, 0, 0.5, 1);
}
"""
)
pr

GLProgramWidget(uid=0)

In [5]:
buf = w.create_buffer()
buf.update(np.array(
    [ 0, 0,
      0, 0.5,
      0.7, 0,
    ], dtype=np.float32))
buf

GLBufferWidget(uid=0)

In [6]:
vao = w.create_vao()
vao.bind(
    pr, [
        (buf, "2f32", "a_position")
    ]
)
vao

GLVertexArrayWidget(uid=0)

In [7]:
w.clear()
w.use_program(pr)
w.bind_vao(vao)
w.draw_arrays('triangles', 0, 3)
w.render()

In [8]:
pr.compile(
"""#version 300 es
 
// an attribute is an input (in) to a vertex shader.
// It will receive data from a buffer
in vec4 a_position;
uniform vec2 u_pos;
 
// all shaders have a main function
void main() {
 
  // gl_Position is a special variable a vertex shader
  // is responsible for setting
  gl_Position = a_position + vec4(u_pos,0,0);
}
"""
,
"""#version 300 es
 
// fragment shaders don't have a default precision so we need
// to pick one. highp is a good default. It means "high precision"
precision highp float;
 
// we need to declare an output for the fragment shader
out vec4 outColor;
 
void main() {
  // Just set the output to a constant reddish-purple
  outColor = vec4(1, 0, 0.5, 1);
}
"""
)
pr

GLProgramWidget(uid=0)

In [9]:
w.clear()
w.use_program(pr)
w.set_uniform('u_pos', np.asarray([0.5,0.5], dtype=np.float32))
w.bind_vao(vao)
w.draw_arrays('triangles', 0, 3)
w.render()

In [10]:
axis_pr = w.create_program()
axis_pr.compile(
"""#version 300 es
uniform mat4 ViewProjection;
uniform mat4 Model;
uniform float Scale;
in vec3 in_vert;
in vec3 in_color;
out vec3 v_color;    
void main() {
    gl_Position = vec4(in_vert * Scale, 1.0) * Model * ViewProjection;
    v_color = in_color;
}
"""
,
"""#version 300 es
precision highp float;

in vec3 v_color;
out vec4 f_color;
void main() {
    f_color = vec4(v_color, 1.0);
}
"""
)
axis_buf = w.create_buffer()
axis_buf.update(np.array(
    [ 
        # x, y ,z red, green, blue
        0, 0, 0, 1, 0, 0,
        1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 1,
    ], dtype=np.float32))
axis_vao = w.create_vao()
axis_vao.bind(
    axis_pr, [
        (axis_buf, '3f32 3f32', 'in_vert', 'in_color')
    ]
)
display(axis_pr, axis_buf, axis_vao)

GLProgramWidget(uid=1)

GLBufferWidget(uid=1)

GLVertexArrayWidget(uid=1)

In [11]:

w.clear()
w.use_program(axis_pr)
w.set_uniform_matrix('Model', np.array([1,0,0,50, 0,1,0,0, 0,0,1,0, 0,0,0,1], dtype=np.float32).reshape([4,4]))
w.set_uniform('Scale', np.array([100], dtype=np.float32))
w.bind_vao(axis_vao)
w.draw_arrays('lines', 0, 6)
w.render()

In [12]:
from ipywidgets import widgets, interact
def move_x(x):
    w.clear()
    w.use_program(axis_pr)
    w.set_uniform_matrix('Model', np.array([1,0,0,x, 0,1,0,0, 0,0,1,0, 0,0,0,1], dtype=np.float32).reshape([4,4]))
    w.set_uniform('Scale', np.array([10], dtype=np.float32))
    w.bind_vao(axis_vao)
    w.draw_arrays('lines', 0, 6)
    w.render()
interact(move_x, x=widgets.IntSlider(min=-100, max=100, step=1, value=0))
w

interactive(children=(IntSlider(value=0, description='x', min=-100), Output()), _dom_classes=('widget-interact…

GLViewer()